# Ejercicios Dataframes 1
Realiza los siguientes ejercicios. Soluciona cada uno de ellos empleando la DataFrame API y Spark SQL
1. Inicializa la variable spark (0)

In [3]:
import string
import sys
from pyspark import sql

spark = sql.SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("03-dfej2") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
    .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
    .getOrCreate()

sc = spark.sparkContext

2. Crea un dataframe llamado mi_df con los datos del archivo *data/retail-data/all/online-retail-dataset.csv*

In [11]:
#DataFrame API
mi_df = spark.read.option("header", True).option("inferSchema","true").csv("/user/jovyan/data/retail-data/all/online-retail-dataset.csv")


#SQL



3. Muestra las 10 primeras filas del dataframe

In [12]:
#DataFrame API
mi_df.show(10)
mi_df.printSchema()
#SQL
mi_df.createOrReplaceTempView("retail")
spark.sql("select * from retail limit 10").show()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

4. Muestra sólo los camos "InvoiceNo", "Description" y "Quantity" de las 5 primeras filas

In [15]:
#DataFrame API
mi_df.select("InvoiceNo", "Description", "Quantity").show()

#SQL
spark.sql("select InvoiceNo, Description, Quantity from retail").show()


+---------+--------------------+--------+
|InvoiceNo|         Description|Quantity|
+---------+--------------------+--------+
|   536365|WHITE HANGING HEA...|       6|
|   536365| WHITE METAL LANTERN|       6|
|   536365|CREAM CUPID HEART...|       8|
|   536365|KNITTED UNION FLA...|       6|
|   536365|RED WOOLLY HOTTIE...|       6|
|   536365|SET 7 BABUSHKA NE...|       2|
|   536365|GLASS STAR FROSTE...|       6|
|   536366|HAND WARMER UNION...|       6|
|   536366|HAND WARMER RED P...|       6|
|   536367|ASSORTED COLOUR B...|      32|
|   536367|POPPY'S PLAYHOUSE...|       6|
|   536367|POPPY'S PLAYHOUSE...|       6|
|   536367|FELTCRAFT PRINCES...|       8|
|   536367|IVORY KNITTED MUG...|       6|
|   536367|BOX OF 6 ASSORTED...|       6|
|   536367|BOX OF VINTAGE JI...|       3|
|   536367|BOX OF VINTAGE AL...|       2|
|   536367|HOME BUILDING BLO...|       3|
|   536367|LOVE BUILDING BLO...|       3|
|   536367|RECIPE BOX WITH M...|       4|
+---------+--------------------+--

5. Muestra sólo los campos "InvoiceNo" (Factura), "Description" (Descripción) y "Quantity" (Cantidad) de lsa 5 primeras filas

In [19]:

#DataFrame API
mi_df.select("InvoiceNo", "Description", "Quantity").show(5)

#SQL
spark.sql("select InvoiceNo, Description, Quantity from retail limit 5").show()





+---------+--------------------+--------+
|InvoiceNo|         Description|Quantity|
+---------+--------------------+--------+
|   536365|WHITE HANGING HEA...|       6|
|   536365| WHITE METAL LANTERN|       6|
|   536365|CREAM CUPID HEART...|       8|
|   536365|KNITTED UNION FLA...|       6|
|   536365|RED WOOLLY HOTTIE...|       6|
+---------+--------------------+--------+
only showing top 5 rows

+---------+--------------------+--------+
|InvoiceNo|         Description|Quantity|
+---------+--------------------+--------+
|   536365|WHITE HANGING HEA...|       6|
|   536365| WHITE METAL LANTERN|       6|
|   536365|CREAM CUPID HEART...|       8|
|   536365|KNITTED UNION FLA...|       6|
|   536365|RED WOOLLY HOTTIE...|       6|
+---------+--------------------+--------+



6. Muestra los campos "Description" y "Quantity" de las 5 primeras filas para el número de factura 536365

In [23]:
#DataFrame API

mi_df.filter("InvoiceNO == 536365").select("Description","Quantity").show(5)
#SQL
spark.sql("Select Description, Quantity from retail where InvoiceNo = 536365 limit 5").show()


+--------------------+--------+
|         Description|Quantity|
+--------------------+--------+
|WHITE HANGING HEA...|       6|
| WHITE METAL LANTERN|       6|
|CREAM CUPID HEART...|       8|
|KNITTED UNION FLA...|       6|
|RED WOOLLY HOTTIE...|       6|
+--------------------+--------+
only showing top 5 rows

+--------------------+--------+
|         Description|Quantity|
+--------------------+--------+
|WHITE HANGING HEA...|       6|
| WHITE METAL LANTERN|       6|
|CREAM CUPID HEART...|       8|
|KNITTED UNION FLA...|       6|
|RED WOOLLY HOTTIE...|       6|
+--------------------+--------+



7. Añade una nueva columna llamada "Mayor_que_20" que contenga el valor True si el coste total (quantity * UnitPrice) es mayor que 20, False en caso contrario. Muestra las 5 primeras filas.

In [30]:
from pyspark.sql.functions import expr
#DataFrame API
mi_df.withColumn("Mayor_que_20",expr("quantity * UnitPrice > 20")).show()

#SQL
spark.sql("SELECT *, quantity* UnitPrice >20 as Mayor_que_20 FROM retail").show()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Mayor_que_20|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|       false|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|        true|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|        true|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|        true|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|        true|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7

8. Elimina la columna CustomerID

In [33]:
#DataFrame API
mi_df.drop("CustomerID").show()

#SQL
spark.sql("SELECT InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, Country from retail").show()


+---------+---------+--------------------+--------+--------------+---------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|       Country|
+---------+---------+--------------------+--------+--------------+---------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.25|United Kingdom|
|   536366|    22633|HAND WARMER UNION...|       6|12/1/2010 8:28|     1.85|Unit

9. Muestra los países distintos (10 primeros)

In [41]:
#DataFrame API
mi_df.select("Country").distinct().show()

#SQL
spark.sql("SELECT distinct Country from retail").show()


+---------------+
|        Country|
+---------------+
|         Sweden|
|      Singapore|
|        Germany|
|            RSA|
|         France|
|         Greece|
|        Belgium|
|        Finland|
|          Malta|
|    Unspecified|
|          Italy|
|           EIRE|
|         Norway|
|          Spain|
|        Denmark|
|      Hong Kong|
|         Israel|
|        Iceland|
|Channel Islands|
|            USA|
+---------------+
only showing top 20 rows

+------------------+
|           Country|
+------------------+
|            Sweden|
|         Singapore|
|           Germany|
|            France|
|            Greece|
|European Community|
|           Belgium|
|           Finland|
|             Malta|
|       Unspecified|
|             Italy|
|              EIRE|
|         Lithuania|
|            Norway|
|             Spain|
|           Denmark|
|         Hong Kong|
|           Iceland|
|            Israel|
|   Channel Islands|
+------------------+
only showing top 20 rows



10. Muestra el número de países distintos

In [43]:
#DataFrame API
print(mi_df.select("Country").distinct().count())

#SQL
spark.sql("SELECT count(distinct Country) from retail").show()


38
+-----------------------+
|count(DISTINCT Country)|
+-----------------------+
|                     38|
+-----------------------+



11. Muestra los países distintos, ordenados de forma ascendente

In [45]:
#DataFrame API
mi_df.select("Country").distinct().sort("Country").show()

#SQL

spark.sql("SELECT distinct Country from retail order by 1").show()

+------------------+
|           Country|
+------------------+
|         Australia|
|           Austria|
|           Bahrain|
|           Belgium|
|            Brazil|
|            Canada|
|   Channel Islands|
|            Cyprus|
|    Czech Republic|
|           Denmark|
|              EIRE|
|European Community|
|           Finland|
|            France|
|           Germany|
|            Greece|
|         Hong Kong|
|           Iceland|
|            Israel|
|             Italy|
+------------------+
only showing top 20 rows

+------------------+
|           Country|
+------------------+
|         Australia|
|           Austria|
|           Bahrain|
|           Belgium|
|            Brazil|
|            Canada|
|   Channel Islands|
|            Cyprus|
|    Czech Republic|
|           Denmark|
|              EIRE|
|European Community|
|           Finland|
|            France|
|           Germany|
|            Greece|
|         Hong Kong|
|           Iceland|
|            Israel|
|       